# 

In [71]:
from mc2hlib.common import load_pdf
from mc2hlib import lh
from rabbit import io_tools
import numpy as np
import copy 
import re
import lhapdf
import pandas as pd
from wremnants import theory_tools

sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
sh: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory


In [2]:
pdf_name = "NNPDF40_nnlo_hessian_pdfas"
Q = 100
max_nf = 5
photon = False

In [132]:
fitresult,meta = io_tools.get_fitresult(
    "/scratch/submit/cms/kdlong/CombineStudies/Unblinded/WMass_eta_pt_charge_nominal/fitresults.hdf5",
    meta=True
)
labels, pulls, constraints = io_tools.get_pulls_and_constraints(
    fitresult,
    keep_nuisances=r"pdf\d+",
)

In [133]:
pdf_input = meta['meta_info_input']['meta_info_input']['args']['pdfs'][0]

In [135]:
pdf_name = theory_tools.pdfMap[pdf_input]["lha_name"]

In [136]:
pdf, fl, xgrid = load_pdf(pdf_name, Q, max_nf, photon)

LHAPDF 6.5.4 loading all 59 PDFs in set CT18ZNNLO
CT18ZNNLO, version 1; 59 PDF members


In [137]:
pdf.n_rep

58

In [262]:
meta["meta_info_input"]['meta_info']['args']['symmetrizePdfUnc']

'quadratic'

In [139]:
meta["meta_info_input"]['meta_info']['args']['scalePdf']

-1.0

In [140]:
pdf_lha = lhapdf.getPDFSet(pdf_name)
pdf_lha

PDFSet<CT18ZNNLO, 58 hessian vars: 34 keys, 29 local>

In [141]:
errors = pdf_lha.errorInfo

In [149]:
nhess = errors.nmemCore

In [150]:
nhess

58

In [148]:
if pulls.size - 1 == nhess:
    pdf_nuis_regex = r"pdf(?![1][^\d])\d+"
    labels, pulls, constraints = io_tools.get_pulls_and_constraints(
    keep_nuisances=pdf_nuis_regex,
)

In [247]:
new_pdf, fl, xgrid = load_pdf(pdf_name, Q, max_nf, photon)

In [248]:
cov = fitresult["cov"].get()
var_names = np.array(cov.axes["parms_x"])

pdf_mask = np.isin(var_names, labels)

cov_pdf = cov.values()[np.ix_(pdf_mask, pdf_mask)]

In [249]:
eigv, V = np.linalg.eigh(cov_pdf)
K = V*np.sqrt(np.maximum(eigv, 0))
cov_pdf.shape

(58, 58)

In [250]:
headers, grids = lh.load_all_replicas(pdf, lhapdf.paths()[0] + "/" + pdf.pdf_name + "/" + pdf.pdf_name)

-> Reading replica from LHAPDF 58 

In [158]:
grids[0].shape

(63756,)

In [251]:
# Don't include alpha_s (the central grid is left off by default)
matrix = lh.big_matrix(grids)

/home/submit/kdlong/mc2hessian/env/lib/python3.12/site-packages/mc2hlib/lh.py:103: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  X = pd.concat(gridlist[1:], axis=1,


In [252]:
new_matrix = matrix.copy()

In [253]:
symm_avg = 0.5*(new_matrix.values[:,::2] + new_matrix.values[:,1::2])
symm_diff = 0.5*np.sqrt(3)*(new_matrix.values[:,::2] - new_matrix.values[:,1::2])

In [266]:
new_matrix.iloc[:,avg_idx] = symm_avg
new_matrix.iloc[:,~avg_idx] = symm_diff

In [265]:
avg_idx = np.char.find(labels, "Avg") != -1

In [255]:
central_grid = grids[0]+np.sum(pulls*matrix, axis=1)

In [256]:
result = matrix.dot(K).add(central_grid, axis=0)

In [258]:
base = lhapdf.paths()[0] + "/" + pdf.pdf_name + "/" + pdf.pdf_name
set_name = ""
folder = "/scratch/submit/cms/wmass/PostfitPDF"
name = "/".join([folder, set_name])

In [259]:
inn = open(base + ".info", 'r')
out = open(folder + "/" + set_name + ".info", 'w')
for l in inn.readlines():
    if l.find("SetDesc:") >= 0: out.write(f"SetDesc: \"{pdf.pdf_name} modified by CMS mW postfit covariance\"\n")
    else: out.write(l)
inn.close()
out.close()

In [260]:
lh.write_replica(0, name, b"PdfType: 'central'\nFormat: lhagrid1\n", central_grid)
for column in result.columns:
    header = b"PdfType: 'error'\nFormat: lhagrid1\n"
    lh.write_replica(column+1, name, header, result[column])